# 6. 프로젝트 : 멋진 작사가 만들기

## 6.1 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Well, summer slipped us underneath her tongue', 'Our days and nights are perfumed with obsession', 'Half of my wardrobe is on your bedroom floor']


## 6.2 데이터 정제

In [2]:
import re
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue

        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> well , summer slipped us underneath her tongue <end>',
 '<start> our days and nights are perfumed with obsession <end>',
 '<start> half of my wardrobe is on your bedroom floor <end>',
 '<start> use our eyes , throw our hands overboard i am your sweetheart psychopathic crush <end>',
 '<start> drink up your movements , still i can t get enough <end>',
 '<start> i overthink your p punctuation use <end>',
 '<start> not my fault , just a thing that my mind do a rush at the beginning <end>',
 '<start> i get caught up , just for a minute <end>',
 '<start> but lover , you re the one to blame , all that you re doing <end>',
 '<start> can you hear the violence ? <end>']

In [4]:
import tensorflow as tf
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=16, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  136    4 ...    0    0    0]
 [   2  155  377 ...    0    0    0]
 [   2  541   19 ...    0    0    0]
 ...
 [   2    5   22 ... 4178    4    3]
 [   2    5   22 ... 4178    4    3]
 [   2    5   22 ... 4178   20    3]] <keras_preprocessing.text.Tokenizer object at 0x7f3603676090>


In [5]:
print(tensor[:3, :15])

[[   2  136    4  584 3121  131 1299   69  969    3    0    0    0    0
     0]
 [   2  155  377    8  858   77 9560   31 8541    3    0    0    0    0
     0]
 [   2  541   19   13 5339   26   18   21 1473  363    3    0    0    0
     0]]


## 6.3 평가 데이터셋 분리

In [6]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2  136    4  584 3121  131 1299   69  969    3    0    0    0    0
    0]
[ 136    4  584 3121  131 1299   69  969    3    0    0    0    0    0
    0]


In [7]:
from sklearn.model_selection import train_test_split


enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                                tgt_input, 
                                                                test_size=0.2,
                                                                shuffle=True)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)



Source Train: (140599, 15)
Target Train: (140599, 15)


1. 학습데이터 갯수가 140599로 권장 데이터인 124960개를 초과하였으나 권장사항이기 때문에 일단 진행

In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1  

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset
val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset

<BatchDataset shapes: ((256, 15), (256, 15)), types: (tf.int32, tf.int32)>

## 6.4 인공지능 만들기

### 6.4.1 모델생성

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2050
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [10]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 15, 12001), dtype=float32, numpy=
array([[[-5.62606983e-05,  7.10867316e-05, -1.09179724e-04, ...,
          5.86278147e-05, -2.24110714e-04, -2.21317852e-04],
        [-3.24620429e-04,  3.16536207e-05,  2.70948687e-04, ...,
          1.17612137e-04, -2.89374788e-04, -8.85770714e-05],
        [-5.18722693e-04, -2.11580482e-04,  4.54634108e-04, ...,
          4.35776106e-04, -4.33672074e-04,  1.00923018e-04],
        ...,
        [-2.01043906e-03, -5.89511881e-04, -1.88707781e-03, ...,
          5.69538504e-04,  1.07571273e-03,  2.61607254e-03],
        [-2.10939022e-03, -6.69227447e-04, -1.98147120e-03, ...,
          7.37804628e-04,  1.11552875e-03,  2.95402901e-03],
        [-2.19090353e-03, -7.55699119e-04, -2.03627162e-03, ...,
          9.16334742e-04,  1.14584470e-03,  3.26304068e-03]],

       [[-5.62606983e-05,  7.10867316e-05, -1.09179724e-04, ...,
          5.86278147e-05, -2.24110714e-04, -2.21317852e-04],
        [ 3.98012344e-04, -1.40746852e-04, -4

In [11]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  21016600  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33628200  
_________________________________________________________________
dense (Dense)                multiple                  24614051  
Total params: 85,403,363
Trainable params: 85,403,363
Non-trainable params: 0
_________________________________________________________________


### 6.4.2 모델 학습

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, validation_data = val_dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 258s 469ms/step - loss: 3.4751 - val_loss: 3.1071
Epoch 2/10
549/549 [==============================] - 258s 470ms/step - loss: 2.9381 - val_loss: 2.8628
Epoch 3/10
549/549 [==============================] - 259s 471ms/step - loss: 2.7090 - val_loss: 2.7147
Epoch 4/10
547/549 [============================>.] - ETA: 0s - loss: 2.5141

model. fit 에서 기존 train dataset을 지정, 추가적인 validation_data를 지정.
사실 이 부분을 tf홈페이지에서 찾아보았으나 찾아내지 못하여 다른 수강생에게 힌트를 얻어 진행하였습니다.

1. embedding_size = 256 hidden_size = 1024 --> 2.5204
2. embedding_size = 256 hidden_size = 512 --> 2.8122 
3. embedding_size = 256 hidden_size = 2048 --> 2.3344
4. embedding_size = 512 hidden_size = 2048 --> 2.3661
5. embedding_size = 512 hidden_size = 1024 --> 2.5845
6. embedding_size = 256 hidden_size = 1500 --> 2.4927
7. embedding_size = 400 hidden_size = 1900 --> 2.2943
8. embedding_size = 400 hidden_size = 1700 --> 2.4264
9. embedding_size = 128 hidden_size = 1900 --> 2.4036
10. embedding_size = 300 hidden_size = 1900 --> 2.3924
11. embedding_size = 400 hidden_size = 2000 --> 4epoch에서 2.6대 나와서 stop
12. maxlen 16으로 수정, embedding_size = 512 hidden_size = 2050

### 6.4.3 모델 평가

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)
